In [3]:
print('hello world')

hello world


In [ ]:
import os
import sys
import csv
from pyspark.sql import SparkSession
from csv import reader
from pyspark.sql.functions import col, explode,split

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [ ]:
spark = init_spark()
movies = spark.read.csv(os.path.join(".","data","movies_metadata.csv"), header=True, inferSchema=True)

# Select columns we are interested in
movies = movies.select("id", "title", "genres").distinct()
printMovie = movies

from pyspark.sql.functions import from_json, col, explode,lower
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType

# Define the schema for the "genres" column
genres_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

# Convert the "genres" column to an array of structs
movies = movies.withColumn("genres", from_json(col("genres"), ArrayType(genres_schema)))

movies = movies.select("id", "title", explode(col("genres")).alias("genre")) \
               .select("id", "title", col("genre.name").alias("genre_name")) \
               .dropna()

# Select all rows with title containing toy story
persuasion_movies = movies.filter(lower(col("title")).like("%persuasion%"))
persuasion_movies.show(truncate=False)

#Training and testing split
(training, testing) = movies.randomSplit([0.8, 0.2], seed=12345)
from pyspark.ml.recommendation import ALS

# # Initialize the ALS model
# als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

# # Fit the model to the training data
# model = als.fit(training)

# # Make predictions on the testing data
# predictions = model.transform(testing)

